In [92]:
import re
lista_strings = ["Olá, mundo!", "Este é um exemplo de string com @caracteres especiais#"]


def limpar_caracteres_especiais(string):

    return re.sub(r'[^\w\s]', '', string)


lista_limpa = [limpar_caracteres_especiais(string) for string in lista_strings]


for string in lista_limpa:
    print(string)

Olá mundo
Este é um exemplo de string com caracteres especiais


In [93]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [94]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\guido\OneDrive\Documentos\GitHub\Cdados


In [95]:
train = pd.read_excel('Dados\dados_treino.xlsx')
train.head(5)

,Target,Mensagem
0,Positivo,"Leitura rápida, com algumas dicas para expandi..."
1,Neutro,"Coaching virou um grande modismo, assim como c..."
2,Negativo,O fundo da história é pra se pensar. Mas é mui...
3,Neutro,A tradução é péssima e diferente do original. ...
4,Negativo,Achei sem graça esse livro. Tópicos de livros ...


In [96]:
test = pd.read_excel('dados/dados_teste.xlsx')
test.head(5)

,Target,Mensagem
0,Neutro,"É incrível como os sebos não tem noção, vendem..."
1,Negativo,Livro sem nenhuma coisa de diferente. Autor ba...
2,Negativo,Bem fraco. Achei muito forçada a ligação com o...
3,Neutro,A caixa veio rasgada. como fazia mais de sete ...
4,Negativo,"Que livro chato, gente quase não consegui term..."


In [97]:
df = pd.read_excel('dados/dados_treino.xlsx')

coluna_especifica = df['Mensagem']



def limpar_caracteres_especiais(string):

    return re.sub(r'[^\w\s]', '', string)


nova_coluna_especifica = [limpar_caracteres_especiais(string) for string in coluna_especifica]
nova_coluna_especifica=[string.lower() for string in nova_coluna_especifica]
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

sw = stopwords.words('portuguese')
for i in range(len(nova_coluna_especifica)):
    nova_coluna_especifica[i] = ' '.join([k for k in nova_coluna_especifica[i].split(" ") if k not in sw])


train.Mensagem=nova_coluna_especifica
train


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guido\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Target,Mensagem
0,Positivo,leitura rápida algumas dicas expandirmos perce...
1,Neutro,coaching virou grande modismo assim criticálo ...
2,Negativo,fundo história pra pensar absurda sei lá prendeu
3,Neutro,tradução péssima diferente original amazon dev...
4,Negativo,achei graça livro tópicos livros ancestrais go...
...,...,...
745,Negativo,conteúdo superficial onde propõe dicas básicas...
746,Negativo,péssimo
747,Negativo,autor confunde tecnicas coaching religiao preg...
748,Negativo,livro superficial escrito impressionar leitore...


In [98]:
def separa_palavra(lista):
    palavras=[]
    for frase in lista:
        palavra=frase.split()
        for e in palavra:
            palavras.append(e)
    palavras_series=pd.Series(palavras)
    tabela_palavras=palavras_series.value_counts()
    return tabela_palavras


In [110]:
def separa_palavra_relativa(lista):
    palavras=[]
    for frase in lista:
        palavra=frase.split()
        for e in palavra:
            palavras.append(e)
    palavras_series=pd.Series(palavras)
    tabela_palavras=palavras_series.value_counts(True)
    return tabela_palavras

In [116]:
train.Target=train.Target.astype('category')
test.Target=test.Target.astype('category')

train_positivo = train.loc[train.Target=='Positivo' , :]
train_negativo = train.loc[train.Target=='Negativo' , :]
train_neutro = train.loc[train.Target=='Neutro' , :]

tabela_train_positivo=separa_palavra(train_positivo.Mensagem)
tabela_train_negativo=separa_palavra(train_negativo.Mensagem)
tabela_train_neutro=separa_palavra(train_neutro.Mensagem)
tabela_palavras=separa_palavra(train.Mensagem)

todas=sum(tabela_palavras.values)
todas_positivas=sum(tabela_train_positivo.values)
todas_negativas=sum(tabela_train_negativo.values)
todas_neutras=sum(tabela_train_neutro.values)




<bound method IndexOpsMixin.value_counts of livro          765
livros         149
autor          130
história       127
bem            120
              ... 
voce             1
consumo          1
conservação      1
tratarse         1
autorais         1
Length: 6409, dtype: int64>


In [101]:
tabela_palavras_relativa=separa_palavra_relativa(train.Mensagem)
tabela_train_positivo_relativa=separa_palavra_relativa(train_positivo.Mensagem)
tabela_train_negativo_relativa=separa_palavra_relativa(train_negativo.Mensagem)
tabela_train_neutro_relativa=separa_palavra_relativa(train_neutro.Mensagem)

In [102]:
def probabilidade_frase(frase):
    
    Positivo=1
    Negativo=1
    Neutro=1
    frase_split=frase.split()
    for i in frase_split:
        if i not in tabela_palavras:
            tabela_palavras[i]=0
        if i not in tabela_train_positivo:
            tabela_train_positivo[i]=0

        
        if i not in tabela_train_negativo:
            tabela_train_negativo[i]=0    
        
        
        if i not in tabela_train_neutro:
            tabela_train_neutro[i]=0 
        
        
        chance_pos=(tabela_train_positivo[i]+1)/(todas+todas_positivas)
        Positivo*=chance_pos
    
        chance_neg=(tabela_train_negativo[i]+1)/(todas+todas_negativas)
        Negativo*=chance_neg

        chance_neu=(tabela_train_neutro[i]+1)/(todas+todas_neutras)
        Neutro*=chance_neu

    
    if Negativo>Positivo and Negativo>Neutro:
        return ('Negativo')
    elif Positivo>Negativo and Positivo>Neutro:
        return ('Positivo')
    else:
        return('Neutro')
    


In [103]:
test

,Target,Mensagem
0,Neutro,"É incrível como os sebos não tem noção, vendem..."
1,Negativo,Livro sem nenhuma coisa de diferente. Autor ba...
2,Negativo,Bem fraco. Achei muito forçada a ligação com o...
3,Neutro,A caixa veio rasgada. como fazia mais de sete ...
4,Negativo,"Que livro chato, gente quase não consegui term..."
...,...,...
245,Neutro,Acabei de comprar O Hobbit (versão Kindle) par...
246,Negativo,Gosto demais dele pela vasta cultura geral e ...
247,Neutro,"FiZ a compra errada, era pra comprar o livro e..."
248,Negativo,Não achei muita graça não 👎🏻


In [122]:
def performance(teste):
    fpos=0
    ppos=0
    fneg=0
    pneg=0
    fneu=0
    pneu=0
    
    target=teste.Target
    
    mensagem=teste.Mensagem
    
    mensagem = [limpar_caracteres_especiais(string) for string in mensagem]
    
    mensagem=[string.lower() for string in mensagem]

    for i in range(len(mensagem)):
        mensagem[i] = ' '.join([k for k in mensagem[i].split(" ") if k not in sw])
    
    for i in range(len(target)):
        classi=probabilidade_frase(mensagem[i])
        if target[i]=='Neutro':
            if classi==target[i]:
                pneu+=1
            else:
                fneu+=1
        if target[i]=='Positivo':
            if classi==target[i]:
                ppos+=1
            else:
                fpos+=1
        if target[i]=='Negativo':
            if classi==target[i]:
                pneg+=1
            else:
                fneg+=1
    porc_fneg=(fneg/(fneg+pneg))*100
    porc_fpos=(fpos/(fpos+ppos))*100
    porc_fneu=(fneu/(fneu+pneu))*100
    porc_pneg=(pneg/(fneg+pneg))*100
    porc_ppos=(ppos/(fpos+ppos))*100
    porc_pneu=(pneu/(fneu+pneu))*100
    acur=((ppos+pneu+pneg)/250)*100
    dic={'Falso negativo':f'{porc_fneg}%','Positivo negativo':f'{porc_pneg}%','Falso positivo':f'{porc_fpos}%','Positivo positivo':f'{porc_ppos}%'
          ,'Falso neutro':f'{porc_fneu}%','Positivo neutro':f'{porc_pneu}%','Acurácia':f'{acur}%'}
    resp=pd.Series(dic)
    return(resp)
resposta=performance(test)
resposta.to_frame()
        





,0
Falso negativo,10.344827586206897%
Positivo negativo,89.65517241379311%
Falso positivo,100.0%
Positivo positivo,0.0%
Falso neutro,17.741935483870968%
Positivo neutro,82.25806451612904%
Acurácia,82.8%
